# Analisis de Similitud de SOPs

#### Importo las librerias 

In [26]:
import numpy as np
import pandas as pd
import os
import smart_open
import gensim
import matplotlib.pyplot as plt
import plotly.express as px
from gensim.models import Doc2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial.distance import cosine


#### Paso los documentos de pdf y docx a txt

In [27]:
%run ../SentenceTransformer/1_transformar_formato.py

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'SentenceTransformer/CorpusIQMS'

## Doc2Vec por documento

#### Generacion del Corpus

En read_corpus tokenizamos y etiquetamos los documentos

In [28]:
def read_corpus(fname, tokens_only=False):
    for document in os.listdir(fname):
        file_path = os.path.join(fname, document)
        with smart_open.open(file_path, encoding="iso-8859-1") as f:
            content = f.read()
            tokens = gensim.utils.simple_preprocess(content)
            if tokens_only:
                yield tokens    
            else:
                yield gensim.models.doc2vec.TaggedDocument(tokens, [document])



corpus = list(read_corpus("txt"))

print(corpus [:2])
print("Documentos leidos: ", len(corpus))

[TaggedDocument(words=['objetivo', 'auditorã', 'as', 'de', 'buenas', 'prã', 'cticas', 'de', 'manufactura', 'gmp', 'buenas', 'prã', 'cticas', 'de', 'distribuciã³n', 'gdp', 'son', 'realizadas', 'frecuentemente', 'para', 'verificar', 'que', 'la', 'fabricaciã³n', 'asã', 'como', 'tambiã', 'el', 'almacenaje', 'distribuciã³n', 'de', 'materiales', 'productos', 'siguen', 'las', 'leyes', 'estã', 'ndares', 'las', 'aplicaciones', 'de', 'autorizaciã³n', 'del', 'mercado', 'las', 'regulaciones', 'de', 'bayer', 'este', 'documento', 'provee', 'los', 'requerimientos', 'mã', 'nimos', 'para', 'la', 'planificaciã³n', 'ejecuciã³n', 'documentaciã³n', 'de', 'las', 'auditorã', 'as', 'gmp', 'gdp', 'iso', 'la', 'auditorã', 'de', 'calidad', 'es', 'uno', 'de', 'los', 'elementos', 'clave', 'del', 'sistema', 'de', 'gestiã³n', 'de', 'cumplimiento', 'de', 'bayer', 'mediante', 'dichas', 'auditorã', 'as', 'se', 'determina', 'el', 'grado', 'de', 'cumplimiento', 'de', 'un', 'proveedor', 'fabricante', 'con', 'los', 'requer

Divido el corpus en un set de entranamiento y uno de testeo

In [29]:
# train_corpus, test_corpus = train_test_split(corpus, test_size=0.2, random_state=42)
# print(len(train_corpus))

Guardo el corpus de entranamiento en un txt

In [30]:
with open("corpus.txt", 'w') as file:
    file.write(str(corpus))

#### Entreno el modelo

In [31]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=400, min_count=1, epochs=60)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [32]:
print(f"La palabra 'objetivo' aparece {model.wv.get_vecattr('objetivo', 'count')} veces en el corpus de entrenamiento.")

La palabra 'objetivo' aparece 1109 veces en el corpus de entrenamiento.


Genero los archivos de vectores y tensores para poder verlo de forma grafica en [TensorFlow](https://projector.tensorflow.org/)

In [33]:
model.save_word2vec_format('doc_tensor.w2v', doctag_vec=True, word_vec=False)
model.save('doc_tensor_model')
%run 00_process_d2v_files.py -i doc_tensor.w2v


with open("auxFolder/metadata.tsv", 'r') as file:
    lineas = file.readlines()

lineas = [linea.replace("*dt_", "", 1) for linea in lineas]

with open("auxFolder/metadata.tsv", 'w') as file:
    file.writelines(lineas)


Directorio:  c:\Users\GPKYR\OneDrive - Bayer\Desktop\Analysis-Documents\Doc2Vec


#### Testeo el modelo

In [34]:
y_true = []
y_pred = []

# print(len(test_corpus))

model = Doc2Vec.load('doc_tensor_model')
i=0
simil_list = []
for doc in corpus:
    vec = model.infer_vector(doc.words)
    simil = model.dv.most_similar([vec], topn=4)
    # simil_elem = [[doc[1][0]]]
    # simil_elem[i].append(simil)
    # simil_list.append(simil_elem)
    # print("\033[92m" + f"  ########################################{doc[1][0]}#########################################" + "\033[0m")
    for similar_doc in simil:
        simil_list.append((similar_doc[0], doc[1][0], similar_doc[1]))
        # if(similar_doc[1] > 0.7):
        #     print(f"  Document Corpus: {similar_doc[0]} | Input Document: {doc[1][0]} | Similarity: {similar_doc[1]:.4f}")

    predicted_label = simil[0][0]
    y_true.append(doc.tags[0])
    y_pred.append(predicted_label)

similar_df = pd.DataFrame(simil_list, columns=['Document 1', 'Document 2', 'Similarity'])
similar_df.sort_values(by=["Similarity"], ascending=False, inplace=True)
similar_df.to_csv('simil_doc_doc2vec.csv', index=False)

#### Graficamos

In [35]:
vectors = pd.read_csv('auxFolder/vectors.tsv', sep='\t', header=None)

metadata = pd.read_csv('auxFolder/metadata.tsv', sep='\t', header=None)

Debemos reducir la dimensionalidad

##### PCA

In [36]:
pca = PCA(n_components=2)

vectors_red = pca.fit_transform(vectors)

In [37]:
x = vectors_red[:, 0]
y = vectors_red[:, 1]

fig = px.scatter(vectors_red, x, y, hover_name=metadata[0], title='Reduccion por PCA')

fig.show()



##### T-SNE

In [38]:
tsne = TSNE(n_components=2, random_state=42, perplexity=4)

vectors_red2 = tsne.fit_transform(vectors)

df = pd.DataFrame(vectors_red2)

df['Name'] = metadata[0]

In [39]:
fig = px.scatter(df, x, y, hover_name='Name', title='Reduccion por T-SNE')

fig.show()

## Pruebo similitud con coseno

In [40]:
csv_lines = []
for doc1 in corpus:
    for doc2 in corpus:
        v1 = model.dv[doc1[1][0]]
        v2 = model.dv[doc2[1][0]]
        sim = 1 - cosine(v1, v2)
        csv_lines.append((doc1[1][0], doc2[1][0], sim))
        if(doc1[1][0] != doc2[1][0] and sim > 0.7):
            print(f"{doc1[1][0]}, {doc2[1][0]}, {sim}")

simil_doc_doc2vec_cosine_df = pd.DataFrame(csv_lines, columns=['Document 1', 'Document 2', 'Similarity'])

simil_doc_doc2vec_cosine = simil_doc_doc2vec_cosine_df.to_csv("simil_doc_doc2vec_cosine.csv", index=False)


C0001.txt, C0002.txt, 0.79151870885833
C0001.txt, C0006.txt, 0.9428852911246921
C0002.txt, C0001.txt, 0.79151870885833
C0002.txt, C0006.txt, 0.806657615470615
C0002.txt, C0017.txt, 0.7481450370411797
C0006.txt, C0001.txt, 0.9428852911246921
C0006.txt, C0002.txt, 0.806657615470615
C0006.txt, C0017.txt, 0.7071008507022433
C0017.txt, C0002.txt, 0.7481450370411797
C0017.txt, C0006.txt, 0.7071008507022433
C0017.txt, C0018.txt, 0.8158802436935577
C0018.txt, C0017.txt, 0.8158802436935577
C0019.txt, G0035.txt, 0.7908907773285955
C0024.txt, C0063.txt, 0.70187417345604
C0026.txt, C0028.txt, 0.7010472347767283
C0028.txt, C0026.txt, 0.7010472347767283
C0028.txt, C0030.txt, 0.7341329061917625
C0028.txt, C0056.txt, 0.706892462033644
C0028.txt, C0064.txt, 0.7182166318054187
C0028.txt, C0093.txt, 0.7057601010690795
C0029.txt, C0064.txt, 0.7073609857959198
C0030.txt, C0028.txt, 0.7341329061917625
C0030.txt, C0085.txt, 0.7068322581624791
C0030.txt, C0093.txt, 0.7064453387154301
C0034.txt, C0062.txt, 0.8

In [41]:
cosine_model = gensim.models.doc2vec.Doc2Vec(vector_size=200, min_count=1, epochs=60)
cosine_model.build_vocab(corpus)
cosine_model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

cosine_model.save_word2vec_format('doc_tensor_cosine.w2v', doctag_vec=True, word_vec=False)

%run 00_process_d2v_files.py -i doc_tensor_cosine.w2v

Directorio:  c:\Users\GPKYR\OneDrive - Bayer\Desktop\Analysis-Documents\Doc2Vec


In [42]:
cosine_model.save('doc_tensor_model_cos')

In [43]:
import pandas as pd

# Step 1: Read the CSV file
df = pd.read_csv('auxFolder/cosine/metadata_c.tsv', header=None)

# Step 2: Modify the desired column
# Assuming the column you want to modify is the first column (index 0)
df[0] = df[0].str.replace(r'^\*dt_', '', regex=True)  # Remove '*dt_' at the start
df[0] = df[0].str.replace(r'.txt\$', '', regex=True)  # Remove '.txt' at the end

# Step 3: Save the modified DataFrame back to a TSV file
df.to_csv('auxFolder/cosine/metadata_c.tsv', index=False, header=False)

In [44]:
vectors_c = pd.read_csv('auxFolder/cosine/vectors_c.tsv', sep='\t', header=None)

metadata_c = pd.read_csv('auxFolder/cosine/metadata_c.tsv', sep='\t', header=None)

In [45]:
pca = PCA(n_components=2)

vectors_red_cos = pca.fit_transform(vectors_c)

x = vectors_red_cos[:, 0]
y = vectors_red_cos[:, 1]

fig = px.scatter(vectors_red_cos, x, y, hover_name=metadata_c[0], title='Reduccion por PCA')

fig.show()

In [46]:
pca_df = pd.DataFrame(vectors_red_cos, columns=['PC1', 'PC2'])  # Create DataFrame for PCA results
pca_df['Metadata'] = metadata_c[0]  # Add metadata to the DataFrame

# Step 4: Save the PCA results to a CSV or TSV file
pca_df.to_csv('auxFolder/cosine/vectors_reduced_pca.csv', index=False)  # Save as CSV

In [49]:
print(simil_doc_doc2vec_cosine_df.columns)

Index(['Document 1', 'Document 2', 'Similarity'], dtype='object')


In [ ]:
fig = px.scatter(simil_doc_doc2vec_cosine_df, x="Document 1", y="Document 2", color="Similarity",
                 title="Document Similarity Scatter Plot",
                 labels={"Document 1": "Document 1", "Document 2": "Document 2", "Similarity": "Similarity Score"},
                 hover_name="Document 2")

# Show the plot
fig.show()

In [48]:
# Step 2: Perform t-SNE
tsne = TSNE(n_components=2, random_state=42)  # You can set random_state for reproducibility
vectors_red_tsne = tsne.fit_transform(vectors_c)

# Step 3: Create a DataFrame with t-SNE results and metadata
tsne_df = pd.DataFrame(vectors_red_tsne, columns=['tSNE1', 'tSNE2'])  # Create DataFrame for t-SNE results
tsne_df['Metadata'] = metadata_c[0]  # Add metadata to the DataFrame

# Step 4: Save the t-SNE results to a CSV or TSV file
tsne_df.to_csv('auxFolder/cosine/vectors_reduced_tsne.csv', index=False)  # Save as CSV
# Alternatively, to save as TSV:
# tsne_df.to_csv('auxFolder/cosine/vectors_reduced_tsne.tsv', sep='\t', index=False)

# Step 5: Plot the t-SNE results
fig = px.scatter(tsne_df, x='tSNE1', y='tSNE2', hover_name='Metadata', title='t-SNE Reduction')
fig.show()

